In [63]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0,'../')

from src.legifrance import LegifranceClient,LegifranceDatabase
database = LegifranceDatabase(path='../data/legifrance/database.db')
# database.create_database(overwrite=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from src.legifrance.orm import Article,Section,LawText
from sqlalchemy import select

select_texts = select(LawText)
texts = database.session.execute(select_texts).all()
texts

[(LawText(legi_id='LEGITEXT000006071785', title=Code minier),),
 (LawText(legi_id='LEGITEXT000006070721', title=Code civil),),
 (LawText(legi_id='LEGITEXT000045476241', title=Code pénitentiaire),),
 (LawText(legi_id='LEGITEXT000006070719', title=Code pénal),),
 (LawText(legi_id='LEGITEXT000006070239', title=Code électoral),),
 (LawText(legi_id='LEGITEXT000006070162', title=Code des communes),),
 (LawText(legi_id='LEGITEXT000006074073', title=Code du tourisme),),
 (LawText(legi_id='LEGITEXT000006071570', title=Code des douanes),),
 (LawText(legi_id='LEGITEXT000006075116', title=Code de l'artisanat),),
 (LawText(legi_id='LEGITEXT000006073984', title=Code des assurances),),
 (LawText(legi_id='LEGITEXT000006074236', title=Code du patrimoine),),
 (LawText(legi_id='LEGITEXT000006071318', title=Code du sport),),
 (LawText(legi_id='LEGITEXT000005634379', title=Code de commerce),),
 (LawText(legi_id='LEGITEXT000023983208', title=Code de l'énergie),),
 (LawText(legi_id='LEGITEXT000006074075', ti

In [65]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006070719')
text = database.session.execute(select_texts).first()[0]
text

LawText(legi_id='LEGITEXT000006070719', title=Code pénal)

In [66]:
select_sections = select(Section).where(Section.lawtext==text)
sections = database.session.execute(select_sections).all()

Test suppression en cascade...

In [67]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006071785')
text = database.session.execute(select_texts).first()[0]
text

LawText(legi_id='LEGITEXT000006071785', title=Code minier)

In [68]:
text.get_articles()

[Article(legi_id='LEGIARTI000006627108', title=None),
 Article(legi_id='LEGIARTI000006627110', title=None),
 Article(legi_id='LEGIARTI000006627112', title=None),
 Article(legi_id='LEGIARTI000006627113', title=None),
 Article(legi_id='LEGIARTI000022964298', title=None),
 Article(legi_id='LEGIARTI000022494495', title=None),
 Article(legi_id='LEGIARTI000022483143', title=None),
 Article(legi_id='LEGIARTI000021948636', title=None),
 Article(legi_id='LEGIARTI000006627120', title=None),
 Article(legi_id='LEGIARTI000006627121', title=None),
 Article(legi_id='LEGIARTI000006627126', title=None),
 Article(legi_id='LEGIARTI000006627128', title=None),
 Article(legi_id='LEGIARTI000006627130', title=None),
 Article(legi_id='LEGIARTI000006627131', title=None),
 Article(legi_id='LEGIARTI000006627132', title=None),
 Article(legi_id='LEGIARTI000006627133', title=None),
 Article(legi_id='LEGIARTI000006627134', title=None),
 Article(legi_id='LEGIARTI000006627135', title=None),
 Article(legi_id='LEGIARTI00

In [69]:
from sqlalchemy import delete

delete_query = delete(LawText).where(LawText.legi_id=='LEGITEXT000006071785')
database.session.execute(delete_query)
database.commit()

In [70]:
select_texts = select(LawText).where(LawText.legi_id=='LEGITEXT000006071785')
database.session.execute(select_texts).first() == None

True

In [71]:
select_articles = select(Article).where(Article.legi_id=='LEGIARTI000006627140')
article = database.session.execute(select_articles).first()[0]
article

TypeError: 'NoneType' object is not subscriptable

In [13]:
client = LegifranceClient()
sections = client.get_lawtext(text)

Code pénal: 0it [00:00, ?it/s]

We check that no double section to create...

In [27]:
articles_1=sections[1].get_articles()
doublons = []

for article in articles_1:
    iso = [a for a in articles_1 if a.legi_id==article.legi_id]
    if len(iso)!=1:
        doublons.append(article)


In [30]:
sections_doublons = [d.section for d in doublons]

sections_doublons[0].articles

[Article(legi_id='LEGIARTI000006419414', title=None),
 Article(legi_id='LEGIARTI000006419415', title=None)]

We check that no section already in database...

In [57]:
section_ids = [s.legi_id for s in sections]
select_existing_sections = select(Section).where(Section.legi_id.in_(section_ids))
existing = database.session.execute(select_existing_sections).all()
print(existing)

[]


We check that no article already in database...

In [58]:
articles = []
for section in sections:
    articles.extend(section.get_articles())

In [59]:
legi_ids = [a.legi_id for a in articles]

existing_article_query = select(Article).where(Article.legi_id.in_(legi_ids))
existing = database.session.execute(existing_article_query).all()
print(existing)

[]


[]